## U-Net

출처 : https://medium.com/@msmapark2/u-net-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-u-net-convolutional-networks-for-biomedical-image-segmentation-456d6901b28a

출처 : https://kuklife.tistory.com/119

출처 : https://www.youtube.com/watch?v=O_7mR4H9WLk

## End-to-End방식의 FCN기반 모델

지금 End-to-End과 FCN이 무엇인가?

## U-Net이란?

Data augmentation??인 생각이 들기 시작한다... ㅎㅎ

일단 왼쪽 contracting Path는 relu함수를 사용해서 VGG를 베이스로 이미지를 줄여준다.

확장하는 Expand path는 데이터의 크기를 확장시킨다.

이유는? augmentation같다...

이 논문의 U-net의 장점 : 적은 데이터로 많은 데이터를 augmentation 가능?

그러면 mask RCNN에 적용이 가능하지 않을까?


## 아키텍처 구조

파란색 부분 = 3 X 3 Convolution(Unpadded) 블록, RELU 사용

빨간색 부분 = 2 X 2 Max pooling(stride = 2)

Contracting Path

<img src = 'https://miro.medium.com/max/875/1*i_MUV1KAoILdjS_u6sAfCw.png'>


# Expansive Path

초록색 부분 = Down-sampling을 수행하는 Max pooling 대신에 Up Convolution으로 사용

파란색 부분 = 3 X 3 Convolution, RELU 사용

회색 부분 = Contracting Path에서 추출된 feature map을 가져와 부착

청록색 부분 = 마지막 단에는 1 X 1 Convolution을 두어 마지막에 cell(세포), membrane(세포막)을 구분 

Expansive Path

<img src = 'https://miro.medium.com/max/875/1*4BVF-6Mdpp-Z_4KBbg2dyQ.png'>

여기서 회색 부분은 Contracting Path에서 사용한 feature map을 그대로 가져와서 사용한다

이유 : 왼쪽의 feature map을 crop해서 오른쪽에 보내게 되는데 그 이유는 pixel에 대한 정보가 소실되는 것을 막기 위함이다. 



## U-Net의 특성

U-Net은 기존의 sliding window 탐색 방식을 사용하지 않습니다.

이유 : Sliding window는 탐색한 부분을 또 다시 탐색하는 방법이므로 시간과 비용을 너무 많이 사용합니다.

U-Net의 patch 탐색 방식

그러나 patch방법을 사용하면 6개의 구간만 나눠줘도 사진의 분석이 가능하기 때문에 시간이 빠릅니다.

U-Net의 장점? 신기한점

Data Augmentation Method(Elastic Deformation)

원래 사진 데이터보다 뒤틀린 부분이 같이 반영된다.

그러면 데이터가 실제 데이터와 비슷하게 만들어 지기 때문에 탐지에 용이하다.

<img src = 'https://d3i71xaburhd42.cloudfront.net/661d005cd8a81547135a36f5362310637375e766/3-Figure3-1.png'>




## Overlap

Method = Mirroring

<img src = 'https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb40i94%2FbtqBFwa1fZq%2FMsCZAxhEnEgu0OdcOUsu5K%2Fimg.jpg'>



## Loss Function

### $P_K (x)$ : the probability that k class is in x pixel

k의 class가 x인 pixel에 있을 확률

$ p_k(x) = \frac{e^{a_k (x)}}{\sum_{k' = 1}^{k} e^{a_{k'}(x)}}$

where $a_k(x)$ : activation in feature channel k at thee pixel position x $\in \Omega$ with $\Omega \subset Z^2$,

k : number of classes, $p_k (x)$ : approximated maximum function

### w(x) : the weight map equation

x인 pixel에 가중치를 얼마나 줄 것인가?

$w_c$ = x의 위치에 해당하는 빈도 수

$ w(x) = w_c(x) + w_0 * e^{- \frac{(d_1(x) + d_2 (x))^2}{2\sigma^2}}$

where $w_c : \Omega$ -> R is the weight map to balance the class frequencies

$d_1 : \Omega$ -> R denotes the distance to the border of the nearest cell

$d_2 : \Omega$ -> R denotes the distance to the border of the second nearest cell

E(x) : Cross Entropy(Energy function)

E = $\sum_{x \in \Omega}$ w(x) log($p_{l(x)}$(x))

Where l : $\Omega$ -> {1, ... K} is the true label of each pixel

w : $\Omega$ -> R is a weight map that gives some pixels more importance in training